*Decision Tree 여러 개를 결합하여 동작하는 알고리즘
*RandomForest와는 다르게 Decistion Tree를 순차적으로 학습해야 한다.
*RandomFrest와는 달리 무작위성이 없으며, 이전 Tree의 오차를 다음 Tree에서 보완와는 방법으로 동작
*깊지않은 Depth의 Tree를 사용하여 속도가 빠름

*장점
 *Overfitting에 강인한 알고리즘
*단점
  *매개변수를 잘 조정해야지만 성능을 낼 수 있다.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import util
import pandas as pd
import numpy as np

df = pd.read_csv('myfile_eng.csv', encoding='euc-kr')
df = df[df.dan < 8]
stnd_cols = ['tms','days','race']
odds_cols = ['dan','yun1','yun2','san','bok']
result_cols = ['1nd','2nd','3nd']

df_x = df.drop(stnd_cols+odds_cols+result_cols, axis=1) #기준 칼럼 삭제
df_y = df['1nd'] #(*)

X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.25, random_state=0) 

*GradientBoostingClassifier 하이퍼파라미터
*n_estimators : default=100
  *몇 개의 트리를 사용할 지 결정
*learning_rate : default=0.1
  *이전 트리의 오차를 보정하는 정도를 조절
  *learning_rate를 작게하면 n_estimators 늘려야 비슷한 성능을 낼 수 있다. 두 인수는 trade-off관계
*max_depth : default=3
  *트리의 최대 깊이 지정
  *모델이 과적합되면 max_depth를 낮춘다.

In [ ]:
model = GradientBoostingClassifier(n_estimators=300, 
                               learning_rate = 0.2,    
                               max_depth=None 
                               )
model.fit(X_train, y_train)
print('score: %.2f' % model.score(X_test, y_test))

In [ ]:
print('feature_importances: ', model.feature_importances_) #feature가 분류에 얼마나 사용됐는지 확인

In [ ]:
#feature_importances_ : feature가 알고리즘에서 얼마나 사용되었는지
df_features = pd.DataFrame([X_train.columns,model.feature_importances_])
util.log(df_features, 'df_features1')
df_features = df_features.T.rename(columns={0:'name', 1:'imp'})
# df_features = df_features.rename(columns={0:'name', 1:'imp'})
util.log(df_features, 'df_features2')

In [ ]:
#feature의 사용빈도가 없는 컬럼을
df_features.imp = df_features.imp.astype('float')
util.log(df_features.imp.describe(), 'df_features.imp.describe()')
df_temp = df_features[df_features.imp < 0.002]
# df_temp2 = df_temp.drop([81,116], axis=0)
util.log(df_temp, 'df_temp')